In [2]:
import pandas as pd

data = pd.read_csv('./train.csv',sep=',',dtype=object)
data.shape

(497819, 4)

In [3]:
data.head(10)

,pair_id,name_1,name_2,is_duplicate
0,1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
1,2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
2,3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
3,4,Powermax Rubber Factory,Co. One,0
4,5,Tress A/S,Longyou Industries Park Zhejiang,0
5,6,Wpt Mexico Sa De Cv,Hopkins Mfg. De Mexico S De Rl De Cv,0
6,7,National Bank Of,Action International,0
7,8,Citibank Korea Inc.,Kyor,0
8,9,Prettl Electric Del Bajio S.A. De C.V.,Bva Inc.,0
9,10,"Beijing Zhongyi Rongda Tech Trading Co., Ltd.",N J International Inc.,0


In [4]:
data['name_1'].values[0:10]

array(['Iko Industries Ltd.', 'Apcotex Industries Ltd.',
       'Rishichem Distributors Pvt., Ltd.', 'Powermax Rubber Factory',
       'Tress A/S', 'Wpt Mexico Sa De Cv', 'National Bank Of',
       'Citibank Korea Inc.', 'Prettl Electric Del Bajio S.A. De C.V.',
       'Beijing Zhongyi Rongda Tech Trading Co., Ltd.'], dtype=object)

In [5]:
data_sub = data.sample(100)

In [6]:
import nltk
nltk.download()
from nltk.tokenize import word_tokenize

from parapply import parapply


data_sub['name_1_tokenized'] = parapply(data_sub['name_1'],lambda x: word_tokenize(text=x),n_jobs=4)
data_sub['name_2_tokenized'] = parapply(data_sub['name_2'],lambda x: word_tokenize(text=x),n_jobs=4)

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


/home/nikolaypavlychev/.local/lib/python3.10/site-packages/parapply/__init__.py:128: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if type(obj) == type(pd.Series()):
/home/nikolaypavlychev/.local/lib/python3.10/site-packages/parapply/__init__.py:128: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if type(obj) == type(pd.Series()):


In [7]:
data_sub

,pair_id,name_1,name_2,is_duplicate,name_1_tokenized,name_2_tokenized
406836,406837,"TOTAL OIL INDIA PRIVATE LIMITED, TOTAL",Versalis S.p.A.,0,"[TOTAL, OIL, INDIA, PRIVATE, LIMITED, ,, TOTAL]","[Versalis, S.p.A, .]"
484366,484367,Asif Rubber Industries (Pvt) Ltd.,Ceeco Industries Inc.,0,"[Asif, Rubber, Industries, (, Pvt, ), Ltd, .]","[Ceeco, Industries, Inc, .]"
436697,436698,Shanghai Mega Capital Industrial Development C...,Kapco Industries Inc.,0,"[Shanghai, Mega, Capital, Industrial, Developm...","[Kapco, Industries, Inc, .]"
485554,485555,Dht Trading Development Ltd.,N S H Trading,0,"[Dht, Trading, Development, Ltd, .]","[N, S, H, Trading]"
362577,362578,Safe Tech Co.,Derby Konveyor Bant Sanve Tic A,0,"[Safe, Tech, Co, .]","[Derby, Konveyor, Bant, Sanve, Tic, A]"
...,...,...,...,...,...,...
274154,274155,Sunshine Logistics,Sunland Logistics,0,"[Sunshine, Logistics]","[Sunland, Logistics]"
93101,93102,7 Cargo Corp. 6000,Galaxy Medicare,0,"[7, Cargo, Corp., 6000]","[Galaxy, Medicare]"
366016,366017,S J Rubber Industries Ltd.,"Eve Rubber Institute Co., Ltd.",0,"[S, J, Rubber, Industries, Ltd, .]","[Eve, Rubber, Institute, Co., ,, Ltd, .]"
315918,315919,Sumitomo Corp. Of America,Acs Americas,0,"[Sumitomo, Corp., Of, America]","[Acs, Americas]"


In [8]:
from nltk.stem import SnowballStemmer 

stemmer = SnowballStemmer("english")

data_sub['name_1_stemmed'] = parapply(data_sub['name_1_tokenized'],lambda x: [stemmer.stem(i) for i in x],n_jobs=4)
data_sub['name_2_stemmed'] = parapply(data_sub['name_2_tokenized'],lambda x: [stemmer.stem(i) for i in x],n_jobs=4)

data_sub

/home/nikolaypavlychev/.local/lib/python3.10/site-packages/parapply/__init__.py:128: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if type(obj) == type(pd.Series()):
/home/nikolaypavlychev/.local/lib/python3.10/site-packages/parapply/__init__.py:128: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if type(obj) == type(pd.Series()):


,pair_id,name_1,name_2,is_duplicate,name_1_tokenized,name_2_tokenized,name_1_stemmed,name_2_stemmed
406836,406837,"TOTAL OIL INDIA PRIVATE LIMITED, TOTAL",Versalis S.p.A.,0,"[TOTAL, OIL, INDIA, PRIVATE, LIMITED, ,, TOTAL]","[Versalis, S.p.A, .]","[total, oil, india, privat, limit, ,, total]","[versali, s.p.a, .]"
484366,484367,Asif Rubber Industries (Pvt) Ltd.,Ceeco Industries Inc.,0,"[Asif, Rubber, Industries, (, Pvt, ), Ltd, .]","[Ceeco, Industries, Inc, .]","[asif, rubber, industri, (, pvt, ), ltd, .]","[ceeco, industri, inc, .]"
436697,436698,Shanghai Mega Capital Industrial Development C...,Kapco Industries Inc.,0,"[Shanghai, Mega, Capital, Industrial, Developm...","[Kapco, Industries, Inc, .]","[shanghai, mega, capit, industri, develop, co....","[kapco, industri, inc, .]"
485554,485555,Dht Trading Development Ltd.,N S H Trading,0,"[Dht, Trading, Development, Ltd, .]","[N, S, H, Trading]","[dht, trade, develop, ltd, .]","[n, s, h, trade]"
362577,362578,Safe Tech Co.,Derby Konveyor Bant Sanve Tic A,0,"[Safe, Tech, Co, .]","[Derby, Konveyor, Bant, Sanve, Tic, A]","[safe, tech, co, .]","[derbi, konveyor, bant, sanv, tic, a]"
...,...,...,...,...,...,...,...,...
274154,274155,Sunshine Logistics,Sunland Logistics,0,"[Sunshine, Logistics]","[Sunland, Logistics]","[sunshin, logist]","[sunland, logist]"
93101,93102,7 Cargo Corp. 6000,Galaxy Medicare,0,"[7, Cargo, Corp., 6000]","[Galaxy, Medicare]","[7, cargo, corp., 6000]","[galaxi, medicar]"
366016,366017,S J Rubber Industries Ltd.,"Eve Rubber Institute Co., Ltd.",0,"[S, J, Rubber, Industries, Ltd, .]","[Eve, Rubber, Institute, Co., ,, Ltd, .]","[s, j, rubber, industri, ltd, .]","[eve, rubber, institut, co., ,, ltd, .]"
315918,315919,Sumitomo Corp. Of America,Acs Americas,0,"[Sumitomo, Corp., Of, America]","[Acs, Americas]","[sumitomo, corp., of, america]","[ac, america]"


In [15]:
data_sub['name_1_stemmed_str'] = parapply(data_sub['name_1_stemmed'],lambda x: ' '.join(x),n_jobs=4)
data_sub['name_2_stemmed_str'] = parapply(data_sub['name_2_stemmed'],lambda x: ' '.join(x),n_jobs=4)


/home/nikolaypavlychev/.local/lib/python3.10/site-packages/parapply/__init__.py:128: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if type(obj) == type(pd.Series()):
/home/nikolaypavlychev/.local/lib/python3.10/site-packages/parapply/__init__.py:128: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if type(obj) == type(pd.Series()):


In [16]:
data_sub

,pair_id,name_1,name_2,is_duplicate,name_1_tokenized,name_2_tokenized,name_1_stemmed,name_2_stemmed,name_1_stemmed_str,name_2_stemmed_str
406836,406837,"TOTAL OIL INDIA PRIVATE LIMITED, TOTAL",Versalis S.p.A.,0,"[TOTAL, OIL, INDIA, PRIVATE, LIMITED, ,, TOTAL]","[Versalis, S.p.A, .]","[total, oil, india, privat, limit, ,, total]","[versali, s.p.a, .]","total oil india privat limit , total",versali s.p.a .
484366,484367,Asif Rubber Industries (Pvt) Ltd.,Ceeco Industries Inc.,0,"[Asif, Rubber, Industries, (, Pvt, ), Ltd, .]","[Ceeco, Industries, Inc, .]","[asif, rubber, industri, (, pvt, ), ltd, .]","[ceeco, industri, inc, .]",asif rubber industri ( pvt ) ltd .,ceeco industri inc .
436697,436698,Shanghai Mega Capital Industrial Development C...,Kapco Industries Inc.,0,"[Shanghai, Mega, Capital, Industrial, Developm...","[Kapco, Industries, Inc, .]","[shanghai, mega, capit, industri, develop, co....","[kapco, industri, inc, .]","shanghai mega capit industri develop co. , ltd .",kapco industri inc .
485554,485555,Dht Trading Development Ltd.,N S H Trading,0,"[Dht, Trading, Development, Ltd, .]","[N, S, H, Trading]","[dht, trade, develop, ltd, .]","[n, s, h, trade]",dht trade develop ltd .,n s h trade
362577,362578,Safe Tech Co.,Derby Konveyor Bant Sanve Tic A,0,"[Safe, Tech, Co, .]","[Derby, Konveyor, Bant, Sanve, Tic, A]","[safe, tech, co, .]","[derbi, konveyor, bant, sanv, tic, a]",safe tech co .,derbi konveyor bant sanv tic a
...,...,...,...,...,...,...,...,...,...,...
274154,274155,Sunshine Logistics,Sunland Logistics,0,"[Sunshine, Logistics]","[Sunland, Logistics]","[sunshin, logist]","[sunland, logist]",sunshin logist,sunland logist
93101,93102,7 Cargo Corp. 6000,Galaxy Medicare,0,"[7, Cargo, Corp., 6000]","[Galaxy, Medicare]","[7, cargo, corp., 6000]","[galaxi, medicar]",7 cargo corp. 6000,galaxi medicar
366016,366017,S J Rubber Industries Ltd.,"Eve Rubber Institute Co., Ltd.",0,"[S, J, Rubber, Industries, Ltd, .]","[Eve, Rubber, Institute, Co., ,, Ltd, .]","[s, j, rubber, industri, ltd, .]","[eve, rubber, institut, co., ,, ltd, .]",s j rubber industri ltd .,"eve rubber institut co. , ltd ."
315918,315919,Sumitomo Corp. Of America,Acs Americas,0,"[Sumitomo, Corp., Of, America]","[Acs, Americas]","[sumitomo, corp., of, america]","[ac, america]",sumitomo corp. of america,ac america


In [25]:
train_texts = list(data_sub['name_1_stemmed_str'].values)+list(data_sub['name_2_stemmed_str'].values)

with open('./train_strings.csv','w') as f:
    for item in train_texts:
        f.writelines(item)
        f.writelines('\n')
    f.close()

In [27]:
import fasttext

print('train samples = ',len(train_texts))
model = fasttext.train_unsupervised(input='./train_strings.csv',model="skipgram", minn=2, maxn=5, dim=300,epoch=1, lr=0.5,thread=10)
model.words[0:10]


train samples =  200


Read 0M words
Number of words:  25
Number of labels: 0
Progress: 

['</s>', '.', 'ltd', 'industri', ',', 'co.', 'inc', 'de', 'logist', 'intern']

100.0% words/sec/thread:  201475 lr:  0.000000 avg.loss:      -nan ETA:   0h 0m 0s


In [28]:
model.get_word_vector("industri")

array([-7.57568632e-04,  1.42093459e-05,  3.18497368e-05,  7.09060812e-04,
        2.34476858e-04,  1.54148307e-04,  7.32053653e-04,  4.12353198e-04,
       -3.02965724e-04, -4.80811519e-04, -4.50106105e-04,  1.18049626e-04,
        6.12906471e-04, -3.37458099e-04,  1.71189895e-04, -2.57525709e-04,
        7.42873017e-05,  1.55171350e-04, -6.45196224e-06, -5.47889562e-04,
       -1.49526735e-04,  1.07049782e-04,  2.68683681e-04,  4.38115385e-04,
       -7.04916776e-04, -2.55624705e-04, -1.36663848e-05,  2.87418632e-04,
       -5.32025260e-05,  6.17765763e-04,  2.02649186e-04, -2.05215274e-05,
        8.17711989e-04,  6.66646694e-04, -1.33869689e-04, -2.51419726e-04,
       -2.21539944e-04, -2.29673358e-04, -3.93923750e-04, -1.85336889e-04,
       -4.62934229e-04,  2.72735575e-04,  4.09112035e-05, -4.91612491e-06,
        3.48641188e-04,  9.96897725e-05,  1.84496421e-05, -7.51145766e-04,
       -2.61352456e-04,  1.01345286e-04,  1.21557539e-04,  1.00088000e-04,
       -2.66883930e-04,  

In [29]:
model.get_sentence_vector('total oil india privat limit , total')

array([-4.38502282e-02,  7.04822689e-03,  1.32390885e-02,  2.38903426e-02,
        2.71009505e-02,  1.73946237e-03,  6.51094038e-03, -2.32373811e-02,
        6.65456951e-02, -1.03426976e-02, -1.28796173e-03,  6.41633645e-02,
        1.49560450e-02,  2.76614614e-02,  1.02843791e-02,  2.66252290e-02,
       -1.32669893e-03, -4.37281616e-02, -5.26166148e-02, -9.98715311e-03,
        3.85668725e-02,  3.75451818e-02,  2.72339885e-03, -1.55784152e-02,
       -3.70291881e-02, -3.28566879e-02,  1.61722153e-02,  2.55750753e-02,
       -1.30317537e-02, -2.42408318e-03,  2.49775127e-03, -3.52889998e-03,
       -1.94640248e-03,  2.03994866e-02, -2.95968130e-02, -6.43748511e-03,
        2.77056023e-02, -3.00175007e-02, -4.29764502e-02,  6.88815257e-03,
       -1.30318236e-02, -2.77331076e-03, -7.63470307e-03, -7.85595085e-03,
        2.29499694e-02, -1.12014869e-02, -3.04978974e-02, -2.67490782e-02,
        7.09787593e-04,  3.59724090e-02, -2.37433109e-02, -1.44985644e-02,
       -2.68767793e-02, -

In [32]:
import numpy as np

def cos_sim(a, b):
	dot_product = np.dot(a, b)
	norm_a = np.linalg.norm(a)
	norm_b = np.linalg.norm(b)
	return dot_product / (norm_a * norm_b)


sentence_m = model.get_sentence_vector('total oil india privat limit , total') 
sentence_h = model.get_sentence_vector('total oil india privat')

print(cos_sim(sentence_m, sentence_h))


0.8676509
